In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, date_format
from pyspark.sql.window import Window

### Домашнее задание:
Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),

("2023-11-21", "Electronics", 110, 13000),

("2023-11-22", "Electronics", 105, 12500),

("2023-11-20", "Clothing", 300, 15000),

("2023-11-21", "Clothing", 280, 14000),

("2023-11-22", "Clothing", 320, 16000),

("2023-11-20", "Books", 150, 9000),

("2023-11-21", "Books", 200, 12000),

("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [4]:
spark = SparkSession.builder.appName("hw3_spark").getOrCreate()
data = [
   ("2023-11-20", "Electronics", 100, 12000),
   ("2023-11-21", "Electronics", 110, 13000),
   ("2023-11-22", "Electronics", 105, 12500),
   ("2023-11-20", "Clothing", 300, 15000),
   ("2023-11-21", "Clothing", 280, 14000),
   ("2023-11-22", "Clothing", 320, 16000),
   ("2023-11-20", "Books", 150, 9000),
   ("2023-11-21", "Books", 200, 12000),
   ("2023-11-22", "Books", 180, 10000)
]
columns = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data, columns)
df = df.withColumn("date", date_format(col("date"), "yyyy-MM-dd"))
window_spec = Window.partitionBy("category").orderBy("date").rowsBetween(-2, 0)
rolling_avg_df = df.withColumn("rolling_avg_revenue", avg("revenue").over(window_spec))
pivot_df = rolling_avg_df.groupBy("category").pivot("date").mean("rolling_avg_revenue")
pivot_df.show()
spark.stop()


+-----------+----------+----------+------------------+
|   category|2023-11-20|2023-11-21|        2023-11-22|
+-----------+----------+----------+------------------+
|      Books|    9000.0|   10500.0|10333.333333333334|
|   Clothing|   15000.0|   14500.0|           15000.0|
|Electronics|   12000.0|   12500.0|           12500.0|
+-----------+----------+----------+------------------+

